In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [5]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def test_da_shit(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

def mean_encoder(df, col, target = 'Sales'):
    Mean_encoded_subject = df.groupby([col])[target].mean().to_dict() 
    return df[col].map(Mean_encoded_subject)

In [8]:
train = pd.read_csv('data/train.csv', parse_dates = True)
stores = pd.read_csv('data/store.csv')

In [20]:
full_raw = pd.merge(train, stores, how='outer', on='Store')

In [22]:
raw = full_raw.Open.loc[(full_raw.Sales == 0)] = 0
raw = full_raw.Open.loc[(full_raw.Sales > 0)] = 1

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
raw = full_raw.loc[~((full_raw.Sales.isnull()) | (full_raw.Store.isnull()))]

In [49]:
raw.Date = pd.to_datetime(raw.Date, format='%Y-%m-%d')
raw.DayOfWeek = raw.Date.dt.weekday + 1

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [50]:
raw_open = raw[~(raw.Open==0)]

In [51]:
customer_means = raw_open.groupby(['Store', 'DayOfWeek']).mean()['Customers']
customer_nulls = raw_open.loc[raw_open['Customers'].isnull()]
for i, row in customer_nulls.iterrows():
    store = row['Store']
    DayOfWeek = row['DayOfWeek']
    mean = customer_means.loc[store, DayOfWeek]
    raw_open.loc[i, 'Customers'] = mean

In [52]:
raw_open.reset_index(inplace = True, drop = True)
null_promo = raw_open.loc[raw_open['Promo'].isnull()]

for i, row in null_promo.iterrows():
    above_and_below = raw_open.loc[i-1, 'Promo'] + raw_open.loc[i+1, 'Promo']
    if above_and_below == 2:
        raw_open.loc[i, 'Promo'] = 1
    elif above_and_below == 0:
        raw_open.loc[i, 'Promo'] = 0
        
null_promo = raw_open.loc[raw_open['Promo'].isnull()]

for i, row in null_promo.iterrows():
    date = str(row['Date'])
    mean = raw_open.groupby('Date').mean().loc[date, 'Promo']
    if mean > 0.9:
        raw_open.loc[i, 'Promo'] = 1
    else:
        raw_open.loc[i, 'Promo'] = 0

In [53]:
holiday_nulls = raw_open.loc[raw_open['StateHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = raw_open.loc[raw_open['Date'] == date, 'StateHoliday'].mode()[0]
    raw_open.loc[i, 'StateHoliday'] = mode
    
holiday_nulls = raw_open.loc[raw_open['SchoolHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = raw_open.loc[raw_open['Date'] == date, 'SchoolHoliday'].mode()[0]
    raw_open.loc[i, 'SchoolHoliday'] = mode

In [170]:
clean_open_stores = raw_open.fillna(value=0)

In [171]:
# Baseline
X = clean_open_stores[['Store', 'DayOfWeek']]
y = clean_open_stores['Sales']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek']).loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

23.534890461313807

In [172]:
# Fixing the outlier (Store #817)
clean_open_stores.Customers.loc[((clean_open_stores.Sales >= 20000) & (clean_open_stores.Customers >= 7000))] = clean_open_stores.loc[(clean_open_stores.Store == 817)].mean()['Customers']

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [173]:
# Encoding Store Type 'b' vs not 'b'
clean_open_stores.loc[(clean_open_stores['StoreType'] == 'b'),'StoreType'] = 1
clean_open_stores.loc[((clean_open_stores['StoreType'] == 'a')
                      |(clean_open_stores['StoreType'] == 'c')
                      |(clean_open_stores['StoreType'] == 'd')),'StoreType'] = 0
clean_open_stores.StoreType = clean_open_stores.StoreType.astype('int64', copy=False)

In [174]:
# Encoding State Holidays
clean_open_stores.loc[((clean_open_stores['StateHoliday'] == 0) | (clean_open_stores['StateHoliday'] == 0.0)),'StateHoliday'] = 0
clean_open_stores.loc[((clean_open_stores['StateHoliday'] == 'a') | (clean_open_stores['StateHoliday'] == 'b') | (clean_open_stores['StateHoliday'] == 'c')),'StateHoliday'] = 1
clean_open_stores.StateHoliday = clean_open_stores.StateHoliday.astype('int64', copy=False)

In [175]:
# Promo2 Length Engineering
clean_open_stores['Promo2Time'] = 2015 - clean_open_stores['Promo2SinceYear']
clean_open_stores.Promo2Time.loc[(clean_open_stores.Promo2Time == 2015)] = 0

In [176]:
# Competition Time Engineering
clean_open_stores['CompetitionTime'] = 2015 - clean_open_stores['CompetitionOpenSinceYear']
clean_open_stores.CompetitionTime.loc[(clean_open_stores.CompetitionTime == 2015)] = 0
engineered_years = clean_open_stores.drop(columns = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear','Promo2SinceWeek', 'Promo2SinceYear'])

In [177]:
# Assortment Encoding
engineered_years.Assortment.value_counts()
engineered_years.loc[(engineered_years['Assortment'] == 'a'),'Assortment'] = 0
engineered_years.loc[(engineered_years['Assortment'] == 'b'),'Assortment'] = 1
engineered_years.loc[(engineered_years['Assortment'] == 'c'),'Assortment'] = 2
engineered_years.Assortment = engineered_years.Assortment.astype('int64', copy=False)

In [178]:
# Competition Distance Engineering Based on Quartiles
engineered_years.loc[((engineered_years['CompetitionDistance'] > 0) & (engineered_years['CompetitionDistance'] <= 710)),'CompetitionDistance'] = 3
engineered_years.loc[((engineered_years['CompetitionDistance'] > 710) & (engineered_years['CompetitionDistance'] <= 2320)),'CompetitionDistance'] = 2
engineered_years.loc[((engineered_years['CompetitionDistance'] > 2320) & (engineered_years['CompetitionDistance'] <= 6880)),'CompetitionDistance'] = 1
engineered_years.loc[((engineered_years['CompetitionDistance'] > 6880) & (engineered_years['CompetitionDistance'] <= 75860)),'CompetitionDistance'] = 0

In [179]:
gg = engineered_years.groupby('Store').sum()
avg_purchase_per_store = gg['Sales'] / gg['Customers']
avg_purchase_per_store = pd.DataFrame(avg_purchase_per_store)
engineered_years = engineered_years.merge(avg_purchase_per_store, how = 'left', on = 'Store')
engineered_years.rename({0: 'AvgPurchase'},axis = 1, inplace = True)

In [180]:
final_set = engineered_years.drop(columns=['PromoInterval', 'Customers'])

In [181]:
sample = final_set.sample(frac = 0.2, random_state = 42)

In [182]:
# Random forest
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

18.578430990441657

In [183]:
# Random forest 2
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

mask = y_train > 1200
X_train = X_train[mask]
y_train = y_train[mask]

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

18.64440832420453

In [184]:
# Random forest 3
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

mask = y_train > 500
X_train = X_train[mask]
y_train = y_train[mask]

for i in range(40,120,5):
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=i,random_state=42)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    actuals = np.array(y_test)
    print(f'{i}: {test_da_shit(actuals, preds)}')

40: 18.635237268998228
45: 18.630090702998594
50: 18.6186550449375
55: 18.619428484888076
60: 18.608626855268138
65: 18.610431321468806
70: 18.61328824369845
75: 18.61259461745202
80: 18.611675889567824
85: 18.599917242762874
90: 18.596008181389955
95: 18.590771181284506
100: 18.59580533170328
105: 18.59648949830963
110: 18.60447153467229
115: 18.607110677606894


In [185]:
# Mean Purchase Engineering Based on Quartiles - DOES NOT HELP
"""
final_set.loc[((final_set['MeanPurchase'] > 0) & (final_set['MeanPurchase'] <= 7.79)),'MeanPurchase'] = 0
final_set.loc[((final_set['MeanPurchase'] > 7.79) & (final_set['MeanPurchase'] <= 9.14)),'MeanPurchase'] = 1
final_set.loc[((final_set['MeanPurchase'] > 9.14) & (final_set['MeanPurchase'] <= 10.79)),'MeanPurchase'] = 2
final_set.loc[((final_set['MeanPurchase'] > 10.79) & (final_set['MeanPurchase'] <= 64.96)),'MeanPurchase'] = 3
"""

"\nfinal_set.loc[((final_set['MeanPurchase'] > 0) & (final_set['MeanPurchase'] <= 7.79)),'MeanPurchase'] = 0\nfinal_set.loc[((final_set['MeanPurchase'] > 7.79) & (final_set['MeanPurchase'] <= 9.14)),'MeanPurchase'] = 1\nfinal_set.loc[((final_set['MeanPurchase'] > 9.14) & (final_set['MeanPurchase'] <= 10.79)),'MeanPurchase'] = 2\nfinal_set.loc[((final_set['MeanPurchase'] > 10.79) & (final_set['MeanPurchase'] <= 64.96)),'MeanPurchase'] = 3\n"

In [186]:
# XGBoost Starting Point
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgbr = xgb.XGBRegressor(max_depth=4,learning_rate=0.05,subsample=0.9,n_estimators=1000,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

22.274149771262525

In [187]:
# XGBoost 2 = no subsample, increased learning rate
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgbr = xgb.XGBRegressor(max_depth=4,learning_rate=0.1,n_estimators=1000,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.761977526710798

In [188]:
# XGBoost 3 = no subsample, increased max depth
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.1,n_estimators=1000,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

18.144127513370915

In [189]:
# XGBoost 4 = no subsample, decreased max depth
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgbr = xgb.XGBRegressor(max_depth=3,learning_rate=0.1,n_estimators=1000,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

22.538720238403954

In [190]:
# XGBoost 4 = max_depth=5, test learning rates
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

for i in range(5,30,5):    
    xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=(i/100),n_estimators=1000,n_jobs=1,random_state=42)
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    actuals = np.array(y_test)
    test_da_shit(actuals, preds)
    print(f'{i/100}: {test_da_shit(actuals, preds)}')

0.05: 19.88254155000993
0.1: 18.144127513370915
0.15: 17.667663169002832
0.2: 17.436614375959362
0.25: 17.393457027331674


In [191]:
# XGBoost 5 = max_depth=5, test learning rates further
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

for i in range(15,50,5):    
    xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=(i/100),n_estimators=1000,n_jobs=1,random_state=42)
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    actuals = np.array(y_test)
    test_da_shit(actuals, preds)
    print(f'{i/100}: {test_da_shit(actuals, preds)}')

0.15: 17.667663169002832
0.2: 17.436614375959362
0.25: 17.393457027331674
0.3: 17.45956217116159
0.35: 17.484324979648672
0.4: 17.496922963113466


KeyboardInterrupt: 

In [192]:
# XGBoost 6 = max_depth=5, learning_rate=0.25 , test n_estimators (1500 gives the best result), takes super long to run, do not repeat
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

for i in range(500,2500,500):    
    xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.25,n_estimators=i,n_jobs=1,random_state=42)
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    actuals = np.array(y_test)
    test_da_shit(actuals, preds)
    print(f'{i}: {test_da_shit(actuals, preds)}')

500: 17.950787921112976
1000: 17.393457027331674
1500: 17.369687524884824
2000: 17.446905026609933


In [ ]:
# XGBoost 7 = max_depth=5, learning_rate=0.3, n_estimators=1500, test subsampling
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

for i in range(8,10,1):    
    xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.3,n_estimators=1500,n_jobs=1,random_state=42,subsample=(i/10))
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    actuals = np.array(y_test)
    test_da_shit(actuals, preds)
    print(f'{i/10}: {test_da_shit(actuals, preds)}')

In [ ]:
# XGBoost 8 = max_depth=5, learning_rate=0.3, n_estimators=1500, test colsample_bytree
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

for i in range(3,9,1):    
    xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.3,n_estimators=1500,n_jobs=1,random_state=42,colsample_bytree=(i/10))
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    actuals = np.array(y_test)
    test_da_shit(actuals, preds)
    print(f'{i/10}: {test_da_shit(actuals, preds)}')

In [193]:
# XGBoost 9 = increased max depth
boost_set = sample.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgbr = xgb.XGBRegressor(max_depth=6,learning_rate=0.25,n_estimators=1500,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

17.769798009180715

In [ ]:
# XGBoost 10 = max_depth=5, learning_rate=0.25, n_estimators=1500 - use all of test data
boost_set = final_set.drop(columns='Date')
X = boost_set.drop(columns='Sales')
y = boost_set.Sales
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
   
xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.25,n_estimators=1500,n_jobs=1,random_state=42)
xgbr.fit(X_train, y_train)
preds = xgbr.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)